In [1]:
import psycopg2 as pg
conn = pg.connect(dbname="docker", user= "root", password= "password", host= "172.17.0.1")

In [3]:
cur = conn.cursor()
cur.execute("SELECT COUNT(DISTINCT movie_id) FROM movies;")
print(cur.fetchall())
cur.close()

[(9742,)]


In [4]:
cur = conn.cursor()
cur.execute('''SELECT s.genre
                FROM   movies m, unnest(string_to_array(m.genre, '|')) s(genre)
                GROUP BY s.genre
                order by count(*) desc
                LIMIT 1;''')
print(cur.fetchall())
cur.close()

[('Drama',)]


In [5]:
cur = conn.cursor()
cur.execute('''SELECT m.title, r.rate FROM (
                    SELECT movie_id, AVG(rating) as rate, count(*)
                    FROM ratings
                    GROUP BY movie_id
                    HAVING COUNT(*) > 10
                    ORDERY BY AVG(rating) DESC
                    LIMIT 10) r JOIN movies m ON m.movie_id=r.movie_id;''')
print(cur.fetchall())
cur.close()

[('Secrets & Lies (1996)', 4.590909090909091), ("Guess Who's Coming to Dinner (1967)", 4.545454545454546), ('Paths of Glory (1957)', 4.541666666666667), ('Streetcar Named Desire, A (1951)', 4.475), ('Celebration, The (Festen) (1998)', 4.458333333333333), ('Ran (1985)', 4.433333333333334), ('Shawshank Redemption, The (1994)', 4.429022082018927), ('His Girl Friday (1940)', 4.392857142857143), ('Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)', 4.333333333333333), ('Hustler, The (1961)', 4.333333333333333)]


In [7]:
#when somebody is "most often rating user" is somebody rating 10 times a day between 1995 and 2000
#a more often rating less often than somebody rating 20 times for the last month?

cur = conn.cursor()
cur.execute('''
WITH time_reference AS (
SELECT min(timestamp) AS first_time, max(timestamp) AS last_time from ratings
)
SELECT m.title, r.timestamp, CASE WHEN r.timestamp = tr.first_time THEN 'first rated movie'
                                  WHEN r.timestamp = tr.last_time THEN 'last rated movie' END
from ratings r join time_reference tr on 1=1 INNER JOIN movies m ON r.movie_id = m.movie_id
where r.timestamp = tr.first_time OR r.timestamp = tr.last_time;
''')
print(cur.fetchall())
cur.close()

[('Copycat (1995)', 828124615, 'first rated movie'), ('Apollo 13 (1995)', 828124615, 'first rated movie'), ('Crimson Tide (1995)', 828124615, 'first rated movie'), ('Die Hard: With a Vengeance (1995)', 828124615, 'first rated movie'), ('Boys on the Side (1995)', 828124615, 'first rated movie'), ('Circle of Friends (1995)', 828124615, 'first rated movie'), ('Disclosure (1994)', 828124615, 'first rated movie'), ('Drop Zone (1994)', 828124615, 'first rated movie'), ('Baby-Sitters Club, The (1995)', 828124615, 'first rated movie'), ('Clear and Present Danger (1994)', 828124615, 'first rated movie'), ('Corrina, Corrina (1994)', 828124615, 'first rated movie'), ('Beverly Hills Cop III (1994)', 828124615, 'first rated movie'), ('Black Beauty (1994)', 828124615, 'first rated movie'), ("City Slickers II: The Legend of Curly's Gold (1994)", 828124615, 'first rated movie'), ('Cliffhanger (1993)', 828124615, 'first rated movie'), ('Englishman Who Went Up a Hill But Came Down a Mountain, The (1995)

In [30]:
#when somebody is "most often rating user" is somebody rating 10 times a day between 1995 and 2000
#a more often rating less often than somebody rating 20 times for the last month?

cur = conn.cursor()
cur.execute('''SELECT user_id, count(*) FROM ratings GROUP BY user_id order by count(*) DESC LIMIT 5;''')
print(cur.fetchall())
cur.close()

[]

In [8]:
cur = conn.cursor()
cur.execute('''SELECT title FROM movies WHERE trim(RIGHT(trim(title),5),')')='1995';''')
print(cur.fetchall())
cur.execute('''SELECT title FROM movies WHERE trim(title) ~ '\(1995\)$';''')
print(cur.fetchall())
cur.close()

[('Toy Story (1995)',), ('Jumanji (1995)',), ('Grumpier Old Men (1995)',), ('Waiting to Exhale (1995)',), ('Father of the Bride Part II (1995)',), ('Heat (1995)',), ('Sabrina (1995)',), ('Tom and Huck (1995)',), ('Sudden Death (1995)',), ('GoldenEye (1995)',), ('American President, The (1995)',), ('Dracula: Dead and Loving It (1995)',), ('Balto (1995)',), ('Nixon (1995)',), ('Cutthroat Island (1995)',), ('Casino (1995)',), ('Sense and Sensibility (1995)',), ('Four Rooms (1995)',), ('Ace Ventura: When Nature Calls (1995)',), ('Money Train (1995)',), ('Get Shorty (1995)',), ('Copycat (1995)',), ('Assassins (1995)',), ('Powder (1995)',), ('Leaving Las Vegas (1995)',), ('Othello (1995)',), ('Now and Then (1995)',), ('Persuasion (1995)',), ('City of Lost Children, The (Cité des enfants perdus, La) (1995)',), ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',), ('Dangerous Minds (1995)',), ('Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',), ('Babe (1995)',), ('Dead Man Walking (1995)',), (